<a href="https://colab.research.google.com/github/fidan-c/human-value-detection/blob/main/Value_Det2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import random
import numpy as np

In [3]:
random.seed(64)
np.random.seed(64)

In [ ]:
# using Conc [SEP] Stance [Sep] Premise as one representation: roberta-base, lower
!pip install datasets evaluate transformers[sentencepiece]
!pip install transformers
!pip install tokenizer
!pip install simpletransformers

In [5]:
# tokenizer and dynamic padding imports
#mport tensorflow as tf
#from transformers import AutoTokenizer,TFAutoModel, DataCollatorWithPadding
import pandas as pd


#from datasets import load_dataset  # was necessary for using Hugging Face transformers to have a dictonary, not needed for "simple transformers".

In [6]:
from simpletransformers.classification import (
    MultiLabelClassificationModel, MultiLabelClassificationArgs
)

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
# load data - not including chinese
# sep for tsv
arguments_training_no_label = pd.read_csv('/content/drive/MyDrive/transformers/arguments-training.tsv', sep='\t') 
arguments_validation_no_label = pd.read_csv('/content/drive/MyDrive/transformers/arguments-validation.tsv', sep='\t')
arguments_test_data = pd.read_table('/content/drive/MyDrive/transformers/arguments-test.tsv')
arguments_test_data_nahjalbalagha= pd.read_table('/content/drive/MyDrive/transformers/arguments-test-nahjalbalagha.tsv')

labels_training = pd.read_csv('/content/drive/MyDrive/transformers/labels-training.tsv', sep='\t').drop('Argument ID', axis=1)
labels_validation= pd.read_csv('/content/drive/MyDrive/transformers/labels-validation.tsv', sep='\t').drop('Argument ID', axis=1)

print(labels_training.shape)
labels_training.head()



arguments_training_no_label['Stance'].replace(['in favour of'],
                       ['in favor of'], inplace=True)

arguments_validation_no_label['Stance'].replace(['in favour of'],
                       ['in favor of'], inplace=True)


arguments_test_data['Stance'].replace(['in favour of'],
                       ['in favor of'], inplace=True)

arguments_test_data_nahjalbalagha['Stance'].replace(['in favour of'],
                       ['in favor of'], inplace=True)



(5393, 20)


In [9]:
# concatenate for one representation: for arguments_training dataset

train_labels = labels_training.values.tolist()

cols_to_concat = ["Conclusion", "Stance", "Premise"]
arguments_training_no_label['Concated'] = arguments_training_no_label[cols_to_concat].apply(lambda x: '</s>'.join(x.astype(str)),
    axis=1)

print(arguments_training_no_label['Concated'][0])

arguments_training_no_label['Concated']= arguments_training_no_label['Concated'].str.lower()
print(arguments_training_no_label['Concated'][0])


train_text = arguments_training_no_label['Concated'].values
train_data = [[t, l] for t, l in zip(train_text, train_labels)]  # see the format needed ["text",[label]]
train_df = pd.DataFrame(train_data)
train_df.columns = ["text", "labels"]


We should ban human cloning</s>in favor of</s>we should ban human cloning as it will only cause huge issues when you have a bunch of the same humans running around all acting the same.
we should ban human cloning</s>in favor of</s>we should ban human cloning as it will only cause huge issues when you have a bunch of the same humans running around all acting the same.


In [10]:
# repeat above for the validation set
val_labels = labels_validation.values.tolist()

arguments_validation_no_label['Concated'] = arguments_validation_no_label[cols_to_concat].apply(lambda x: '</s>'.join(x.astype(str)),
    axis=1)

print(arguments_validation_no_label['Concated'][0])

arguments_validation_no_label['Concated']= arguments_validation_no_label['Concated'].str.lower()
print(arguments_validation_no_label['Concated'][0])

val_text = arguments_validation_no_label['Concated'].values

val_data = [[t, l] for t, l in zip(val_text, val_labels)] 
val_df = pd.DataFrame(val_data)
val_df.columns = ["text", "labels"]

Entrapment should be legalized</s>in favor of</s>if entrapment can serve to more easily capture wanted criminals, then why shouldn't it be legal?
entrapment should be legalized</s>in favor of</s>if entrapment can serve to more easily capture wanted criminals, then why shouldn't it be legal?


In [11]:
# repeat above for the main test set

arguments_test_data['Concated'] = arguments_test_data[cols_to_concat].apply(lambda x: '</s>'.join(x.astype(str)),
    axis=1)
print(arguments_test_data['Concated'][0])

arguments_test_data['Concated']= arguments_test_data['Concated'].str.lower()
print(arguments_test_data['Concated'][0])

test_data = arguments_test_data['Concated'].values
np.shape(test_data)


We should end affirmative action</s>against</s>affirmative action helps with employment equity.
we should end affirmative action</s>against</s>affirmative action helps with employment equity.


(1576,)

In [12]:
# repeat above for the secondary test set

arguments_test_data_nahjalbalagha['Concated'] = arguments_test_data_nahjalbalagha[cols_to_concat].apply(lambda x: '</s>'.join(x.astype(str)),
    axis=1)
print(arguments_test_data_nahjalbalagha['Concated'][0])

arguments_test_data_nahjalbalagha['Concated']= arguments_test_data_nahjalbalagha['Concated'].str.lower()
print(arguments_test_data_nahjalbalagha['Concated'][0])

test_data_nahjalbalagha = arguments_test_data_nahjalbalagha['Concated'].values
np.shape(test_data_nahjalbalagha)

Habitual greed devalues humans</s>in favor of</s>Greed is like a slippery rock on which even the steps of scholars are not firm.
habitual greed devalues humans</s>in favor of</s>greed is like a slippery rock on which even the steps of scholars are not firm.


(279,)

In [13]:
model_args = MultiLabelClassificationArgs(num_train_epochs=5, train_batch_size=16, overwrite_output_dir=True)

# Create a MultiLabelClassificationModel from simple transformers: <https://simpletransformers.ai/docs/multi-label-classification/>
model = MultiLabelClassificationModel(
    "roberta",
    "roberta-large",
    num_labels=len(labels_training.columns),
    args=model_args,
)

# Train the model
model.train_model(train_df)

Some weights of the model checkpoint at roberta-large were not used when initializing RobertaForMultiLabelSequenceClassification: ['lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaForMultiLabelSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForMultiLabelSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForMultiLabelSequenceClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.dense.weight', 'classifier.out_proj.bias', 

  0%|          | 0/5393 [00:00<?, ?it/s]

Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Running Epoch 0 of 5:   0%|          | 0/338 [00:00<?, ?it/s]

Running Epoch 1 of 5:   0%|          | 0/338 [00:00<?, ?it/s]

Running Epoch 2 of 5:   0%|          | 0/338 [00:00<?, ?it/s]

Running Epoch 3 of 5:   0%|          | 0/338 [00:00<?, ?it/s]

Running Epoch 4 of 5:   0%|          | 0/338 [00:00<?, ?it/s]

(1690, 0.26556394170286385)

In [13]:
#for LRAP
#result, model_outputs, wrong_predictions = model.eval_model(val_df)

  0%|          | 0/1896 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/237 [00:00<?, ?it/s]

In [14]:
#result

{'LRAP': 0.7541155215063547, 'eval_loss': 0.3092363501902874}

In [14]:
from sklearn.metrics import f1_score




In [19]:
val_df.head()

,text,labels
0,entrapment should be legalized</s>in favor of<...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ..."
1,the use of public defenders should be mandator...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,payday loans should be banned</s>in favor of</...,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."
3,surrogacy should be banned</s>against</s>surro...,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,entrapment should be legalized</s>against</s>e...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ..."


In [53]:
type(list(val_df['text']))

list

In [54]:
# predicted labels and true labels for the validation set for F1 score

# f1_score functions takes np.array

#text = val_df['text'].to_list()

preds, _ = model.predict(list(val_df['text']))
true_labels =np.array(val_labels) 
preds= np.array(preds)

# F1 score for each label separately
f1_scores = f1_score(true_labels, preds, average=None)

# average F1 score across all labels
average_f1_score = np.mean(f1_scores)


  0%|          | 0/1896 [00:00<?, ?it/s]

  0%|          | 0/237 [00:00<?, ?it/s]

In [55]:
average_f1_score

0.4590857993694638

In [59]:
f1_scores

array([0.54373522, 0.60423634, 0.20238095, 0.45      , 0.65527066,
       0.29223744, 0.4375    , 0.18867925, 0.75595621, 0.62032086,
       0.48192771, 0.58088235, 0.19178082, 0.07042254, 0.60204082,
       0.33253012, 0.69177556, 0.7257384 , 0.25742574, 0.496875  ])

In [40]:
print(true_labels.shape)
print(preds.shape)

(1896, 20)
(1896, 20)


In [ ]:
# prediction for the main test set
test_preds = model.predict(list(test_data))

In [ ]:
# prediction outputs for the main test set
cols = ["Argument ID"] + list(labels_validation.columns)
pred_data = []

for arg_id, labels in zip(arguments_test_data["Argument ID"].values, test_preds[0]):
  row = [arg_id] + labels[:]
  pred_data.append(row)

pred_df = pd.DataFrame(pred_data, columns=cols)
pred_df.to_csv("main_test_pred.tsv", index=False, sep='\t')

In [ ]:
# predictions for the nahjalbalagha test set
test_preds_nahjalbalagha= model.predict(list(test_data_nahjalbalagha))

  0%|          | 0/279 [00:00<?, ?it/s]

  0%|          | 0/35 [00:00<?, ?it/s]

In [ ]:
# prediction outputs for the nahjalbalagha test set
cols = ["Argument ID"] + list(labels_validation.columns)
pred_data_nah = []

for arg_id, labels in zip(arguments_test_data_nahjalbalagha["Argument ID"].values, test_preds_nahjalbalagha[0]):
  row = [arg_id] + labels[:]
  pred_data_nah.append(row)

pred_nah_df = pd.DataFrame(pred_data_nah, columns=cols)
pred_nah_df.to_csv("nahjalbalagha_test_pred.tsv", index=False, sep='\t')